# Modelado tabular con Autgluon

In [1]:
!sudo apt-get update
!sudo apt-get install gcsfuse

Hit:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:2 https://download.docker.com/linux/debian bullseye InRelease   
Hit:3 https://deb.debian.org/debian bullseye InRelease              
Hit:4 https://deb.debian.org/debian-security bullseye-security InRelease
Hit:5 https://deb.debian.org/debian bullseye-updates InRelease
Hit:6 https://deb.debian.org/debian bullseye-backports InRelease
Hit:7 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:8 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Hit:9 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Hit:10 https://packages.cloud.google.com/apt google-fast-socket InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcsfuse is already the newest version (3.1.0).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [2]:
#!pip install autogluon.tabular

# Carga librerías

In [2]:
from autogluon.tabular import TabularPredictor
import pandas as pd

In [3]:
!mkdir -p /home/jupyter/franco_maestria/gcs_model_dir_con_prophet_v3

In [4]:
!fusermount -u /home/jupyter/franco_maestria/gcs_model_dir_con_prophet_v3

In [5]:
!gcsfuse forecasting_customer_product /home/jupyter/franco_maestria/gcs_model_dir_con_prophet_v3

{"timestamp":{"seconds":1752849064,"nanos":712021971},"severity":"INFO","message":"Start gcsfuse/3.1.0 (Go version go1.24.0) for app \"\" using mount point: /home/jupyter/franco_maestria/gcs_model_dir_con_prophet_v3\n"}
{"timestamp":{"seconds":1752849064,"nanos":712457993},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"DisableAutoconfig":false,"EnableAtomicRenameObject":true,"EnableGoogleLibAuth":false,"EnableHns":true,"EnableNewReader":false,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":200,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"ExperimentalExcludeRegex":"","ExperimentalParallelDownloadsDefaultOn":true,"MaxParallelDownloads":96,"MaxSizeMb":-1,"ParallelDownloadsPerFile":16,"WriteBufferSize":4194304},"FileSystem":{"DirMode":"755","DisableParallelDirops":false,"ExperimentalEnableDentryCache":false,"ExperimentalEnable

In [6]:
# ------------------------
# 1) Cargar parquet con FE
# ------------------------
import pandas as pd

parquet_path = "panel_cliente_producto_fe.parquet"
df_modelo = pd.read_parquet(parquet_path)
print(f"✅ Parquet cargado. Shape: {df_modelo.shape}")

# ------------------------
# 2) Cargar Prophet features
# ------------------------
csv_path = "prophet_features_customer_product.csv"
df_prophet = pd.read_csv(csv_path)
print(f"✅ CSV Prophet cargado. Shape: {df_prophet.shape}")

# ------------------------
# 3) Asegurar consistencia de tipos
# ------------------------
df_modelo['fecha'] = pd.to_datetime(df_modelo['fecha'])
df_prophet['fecha'] = pd.to_datetime(df_prophet['fecha'])

# ------------------------
# 4) Realizar join por 'product_id' y 'fecha'
# ------------------------
df_modelo_final = df_modelo.merge(
    df_prophet,
    on=['customer_id','product_id', 'fecha'],
    how='left'
)
print(f"✅ Merge completado. Shape final: {df_modelo_final.shape}")

# ------------------------
# 5) Convertir columnas float64 a float32 para ahorrar memoria
# ------------------------
float64_cols = df_modelo_final.select_dtypes(include=['float64']).columns.tolist()

df_modelo_final[float64_cols] = df_modelo_final[float64_cols].astype('float32')

print(f"✅ Conversión de float64 a float32 completada para columnas: {float64_cols}")
print(df_modelo_final.info())

print(f"✅ Merge completado. Shape final: {df_modelo_final.shape}")

# Verifica el resultado
df_modelo_final.head()


✅ Parquet cargado. Shape: (12138186, 194)
✅ CSV Prophet cargado. Shape: (7249573, 9)
✅ Merge completado. Shape final: (12138186, 200)
✅ Conversión de float64 a float32 completada para columnas: ['tn_x', 'inflacion', 'cambio_dolar', 'stock_final', 'clase', 'tn_1', 'diff_tn_1', 'tn_2', 'diff_tn_2', 'tn_3', 'diff_tn_3', 'tn_4', 'diff_tn_4', 'tn_5', 'diff_tn_5', 'tn_6', 'diff_tn_6', 'tn_7', 'diff_tn_7', 'tn_8', 'diff_tn_8', 'tn_9', 'diff_tn_9', 'tn_10', 'diff_tn_10', 'tn_11', 'diff_tn_11', 'tn_12', 'diff_tn_12', 'tn_13', 'diff_tn_13', 'tn_14', 'diff_tn_14', 'tn_15', 'diff_tn_15', 'tn_16', 'diff_tn_16', 'tn_17', 'diff_tn_17', 'tn_18', 'diff_tn_18', 'tn_19', 'diff_tn_19', 'tn_20', 'diff_tn_20', 'tn_21', 'diff_tn_21', 'tn_22', 'diff_tn_22', 'tn_23', 'diff_tn_23', 'tn_24', 'diff_tn_24', 'tn_25', 'diff_tn_25', 'tn_26', 'diff_tn_26', 'tn_27', 'diff_tn_27', 'tn_28', 'diff_tn_28', 'tn_29', 'diff_tn_29', 'tn_30', 'diff_tn_30', 'tn_31', 'diff_tn_31', 'tn_32', 'diff_tn_32', 'tn_33', 'diff_tn_33', 'tn

,customer_id,product_id,periodo,tn_x,IPC,inflacion,cambio_dolar,dias_feriados,cat1,cat2,...,brand_factorized,descripcion_factorized,product_target_enc,customer_target_enc,tn_y,trend,seasonal,additive_terms,residual,slope_trend_3
0,10001,20001,201701,99.438606,102,2.000000,16.080000,1,HC,ROPA LAVADO,...,0,0,2.882843,4.146726,99.438606,110.691025,0.0,-46.415291,35.162876,NaN
1,10001,20001,201702,198.843643,104,1.960784,15.800000,2,HC,ROPA LAVADO,...,0,0,2.882843,4.146726,198.843643,114.229187,0.0,92.249763,-7.635300,NaN
2,10001,20001,201703,92.465370,107,2.884615,15.645000,1,HC,ROPA LAVADO,...,0,0,2.882843,4.146726,92.465370,117.424942,0.0,85.674156,-110.633736,NaN
3,10001,20001,201704,13.297280,109,1.869159,15.490000,1,HC,ROPA LAVADO,...,0,0,2.882843,4.146726,13.297280,120.963104,0.0,-63.645004,-44.020821,3.424027
4,10001,20001,201705,101.005630,111,1.834862,16.184999,2,HC,ROPA LAVADO,...,0,0,2.882843,4.146726,101.005630,124.387131,0.0,-23.848351,0.466848,3.385982


In [7]:
# ------------------------
# 2) Separar en entrenamiento y test
#     👉 Train: periodos <= 201910
#     👉 Test:  periodo = 201912  (input para predecir mes+2)
# ------------------------

train_set = df_modelo_final[(df_modelo_final['fecha'] <= '2019-10-01') & df_modelo_final['clase'].notnull()].copy()
test_set = df_modelo_final[(df_modelo_final['fecha'] == '2019-12-01')].copy()

print(f"Train shape: {train_set.shape} | Test shape: {test_set.shape}")

Train shape: (10297990, 200) | Test shape: (333840, 200)


In [8]:
# ------------------------
# 5) Crear sample_weight basado en tn_total
# ------------------------

# Escalar magnitud de toneladas vendidas
train_set['sample_weight'] = train_set['tn_x']

In [9]:
# ------------------------
# 6) Validar features finales
# ------------------------

features = [col for col in df_modelo_final.columns if col not in [
    'periodo', 'clase', 'tn_y'
]]

print(f"Total features: {len(features)}")
print(features[:10])  # Algunos de ejemplo


Total features: 197
['customer_id', 'product_id', 'tn_x', 'IPC', 'inflacion', 'cambio_dolar', 'dias_feriados', 'cat1', 'cat2', 'cat3']


In [ ]:
# -------------------------------
# 4) Entrenar Autogluon Tabular
# -------------------------------
predictor = TabularPredictor(
    label='clase',
    problem_type='regression',
    eval_metric='mae',
    path='gcs_model_dir_con_prophet'
)

predictor.fit(
    train_data=train_set[features + ['clase']],
    time_limit=14400,
     ag_args_fit={'sample_weight': 'sample_weight'},
    presets='best_quality',
    num_bag_folds=5,
    num_stack_levels=4
)

print("✅ Entrenamiento finalizado.")

# ---------------------------
# 4) Predicciones sobre el test set
# ---------------------------

test_set['tn_pred'] = predictor.predict(test_set[features])
test_set['tn_pred'] = test_set['tn_pred'].clip(lower=0)

# ---------------------------
# 5) Agregar predicciones por producto
# ---------------------------

df_final = (
    test_set.groupby('product_id', as_index=False)['tn_pred']
    .sum()
    .rename(columns={'tn_pred': 'tn'})
)

print(df_final.head())

# ---------------------------
# 6) Exportar CSV final
# ---------------------------

df_final.to_csv('forecast_customer_producto_agregado_prophet_202002_v3.csv', index=False)

print("✅ Archivo generado: forecast_customer_producto_agregado_prophet_202002_v3.csv")
print(f"Productos únicos: {df_final['product_id'].nunique()}")
print(f"Total TN predichas (sum): {df_final['tn'].sum():,.2f}")


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          48
Memory Avail:       264.86 GB / 377.89 GB (70.1%)
Disk Space Avail:   1048576.00 GB / 1048576.00 GB (100.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=4, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

(_ray_fit pid=1661149) [1000]	valid_set's l1: 0.0832147
(_ray_fit pid=1661149) [2000]	valid_set's l1: 0.0822347


(_ray_fit pid=1661149) 	Ran out of time, early stopping on iteration 2035. Best iteration is: [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=1661149) 	[2032]	valid_set's l1: 0.0822004 [repeated 4x across cluster]
(_dystack pid=1655574) 	-0.0828	 = Validation score   (-mean_absolute_error)
(_dystack pid=1655574) 	633.5s	 = Training   runtime
(_dystack pid=1655574) 	68.44s	 = Validation runtime
(_dystack pid=1655574) Fitting model: LightGBM_BAG_L1 ... Training model for up to 100.09s of the 2631.55s of remaining time.
(_dystack pid=1655574) 	Memory not enough to fit 5 folds in parallel. Will train 4 folds in parallel instead (Estimated 18.14% memory usage per fold, 72.54%/80.00% total).
(_dystack pid=1655574) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFitt

(_ray_fit pid=1670366) [1000]	valid_set's l1: 0.0829005


(_ray_fit pid=1670366) 	Ran out of time, early stopping on iteration 1977. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=1670366) 	[1977]	valid_set's l1: 0.0817508 [repeated 4x across cluster]
(_dystack pid=1655574) 	-0.0827	 = Validation score   (-mean_absolute_error)
(_dystack pid=1655574) 	636.05s	 = Training   runtime
(_dystack pid=1655574) 	66.01s	 = Validation runtime
(_dystack pid=1655574) Fitting model: LightGBM_BAG_L2 ... Training model for up to 87.56s of the 1697.55s of remaining time.
(_dystack pid=1655574) 	Memory not enough to fit 5 folds in parallel. Will train 4 folds in parallel instead (Estimated 18.88% memory usage per fold, 75.52%/80.00% total).
(_dystack pid=1655574) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=12, gpus=0, memory=18.88%)
(_dystack pid=1655574) 	Time limit exceeded... Skipping LightGBM_BAG_L2.
(_ray_fit pid=1671339) 	Warning: Model has no time left to train, skipping mode

(_ray_fit pid=1673453) [1000]	valid_set's l1: 0.0838709


(_ray_fit pid=1673453) 	Ran out of time, early stopping on iteration 1401. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=1673453) 	[1401]	valid_set's l1: 0.0833923 [repeated 4x across cluster]
(_dystack pid=1655574) 	-0.0833	 = Validation score   (-mean_absolute_error)
(_dystack pid=1655574) 	474.6s	 = Training   runtime
(_dystack pid=1655574) 	40.18s	 = Validation runtime
(_dystack pid=1655574) Fitting model: LightGBM_BAG_L3 ... Training model for up to 45.11s of the 798.55s of remaining time.
(_dystack pid=1655574) 	Memory not enough to fit 5 folds in parallel. Will train 4 folds in parallel instead (Estimated 18.83% memory usage per fold, 75.33%/80.00% total).
(_dystack pid=1655574) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=12, gpus=0, memory=18.83%)
(_ray_fit pid=1680371) 	Warning: Model has no time left to train, skipping model... (Time Left = -15.9s)
(_dystack pid=1655574) 	Time limit exceeded... Sk

✅ Entrenamiento finalizado.
   product_id           tn
0       20001  1331.555054
1       20002  1066.198364
2       20003   758.457703
3       20004   493.570862
4       20005   487.704651
✅ Archivo generado: forecast_customer_producto_agregado_prophet_202002_v3.csv
Productos únicos: 780
Total TN predichas (sum): 26,054.17


In [13]:
# -------------------------------
# 9) Leaderboard (performance interna)
# -------------------------------
print("\n🔍 Leaderboard:")
lb = predictor.leaderboard(silent=True)
print(lb)

# -------------------------------
# 10) Importancia de features
# -------------------------------
print("\n🔍 Importancia de Features:")
fi = predictor.feature_importance(train_set[features + ['clase']])
fi = fi.reset_index().rename(columns={'index': 'feature'})
print(fi.head(100))


🔍 Leaderboard:
                  model  score_val          eval_metric  pred_time_val  \
0   WeightedEnsemble_L6  -0.080583  mean_absolute_error     444.839525   
1   WeightedEnsemble_L4  -0.081157  mean_absolute_error     368.973023   
2   WeightedEnsemble_L3  -0.081158  mean_absolute_error     260.967582   
3     LightGBMXT_BAG_L3  -0.081286  mean_absolute_error     358.945425   
4   WeightedEnsemble_L2  -0.081294  mean_absolute_error     141.455906   
5     LightGBMXT_BAG_L2  -0.081450  mean_absolute_error     249.581079   
6     LightGBMXT_BAG_L1  -0.081487  mean_absolute_error     119.396747   
7     LightGBMXT_BAG_L4  -0.081520  mean_absolute_error     444.643486   
8   WeightedEnsemble_L5  -0.081520  mean_absolute_error     444.829876   
9     LightGBMXT_BAG_L5  -0.082191  mean_absolute_error     475.750342   
10      LightGBM_BAG_L3  -0.082867  mean_absolute_error     270.613960   
11      LightGBM_BAG_L2  -0.082898  mean_absolute_error     152.445604   
12      LightGBM_BAG_L

These features in provided data are not utilized by the predictor and will be ignored: ['tn_34', 'diff_tn_34', 'tn_35', 'diff_tn_35', 'tn_36', 'diff_tn_36', 'rollmean_1', 'diff_rollmean_1', 'rollmean_34', 'diff_rollmean_34', 'rollmean_35', 'diff_rollmean_35', 'rollmean_36', 'diff_rollmean_36']
Computing feature importance via permutation shuffling for 183 features using 5000 rows with 5 shuffle sets...
	3658.42s	= Expected runtime (731.68s per shuffle set)
	843.08s	= Actual runtime (Completed 5 of 5 shuffle sets)


            feature  importance    stddev   p_value  n  p99_high   p99_low
0             trend    0.035034  0.007000  0.000182  5  0.049447  0.020620
1              tn_x    0.013414  0.004916  0.001825  5  0.023536  0.003292
2    additive_terms    0.011871  0.003216  0.000588  5  0.018493  0.005248
3              tn_1    0.008539  0.002767  0.001156  5  0.014236  0.002842
4     slope_trend_3    0.007285  0.001732  0.000357  5  0.010852  0.003718
..              ...         ...       ...       ... ..       ...       ...
95            tn_17    0.000507  0.000553  0.054913  5  0.001646 -0.000632
96            tn_27    0.000496  0.000223  0.003802  5  0.000955  0.000037
97       diff_tn_25    0.000495  0.000666  0.086037  5  0.001867 -0.000877
98        inflacion    0.000492  0.000293  0.009901  5  0.001095 -0.000111
99  cluster_x_month    0.000484  0.000538  0.057166  5  0.001591 -0.000623

[100 rows x 7 columns]


In [15]:
# Ahora, para guardar 'fi' en un archivo CSV:
fi.to_csv('importancia_feature_modelo_customer_product_18-07-25_Autogluon_Tabular.csv', index=False)